In [5]:
import csv
import json
import math
from operator import itemgetter

#Задание 1

#Парсим данные из CSV
cday = []
with open("context_day.csv") as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        cday.append(row)
cplace = []
with open("context_place.csv") as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        cplace.append(row)
data = []
with open("data.csv") as csvfile:
    reader = csv.reader(csvfile)
    for row in reader:
        data.append(row)

final={}
final["kNN"]={} #Массив конечных результатов
res={} #Заготовка для задания 2

#Вычисление средних оценок
srr=[]#Масив средних оценок у пользователей

for i in range(1,41):   
    sr=0 #Среднее значение
    c=0 #Счётчик для среднего значения
    for m in range(1,31):
        if(int(data[i][m])!=-1):
            c=c+1
            sr+=int(data[i][m])
    srr.append(round(sr/c,3))

for i in range(1,41):
    #Составление метрики сходства
    sim=[]
    for z in range(1,41):
        sumuv=0 # Вверхняя сумма 
        sumuu=0 # Нижняя сумма 1
        sumvv=0 # Нижняя сумма 2
        if(z!=i):
            for m in range(1,31):
                if(int(data[i][m])!=-1 and int(data[z][m])!=-1):
                    sumuv=sumuv+int(data[i][m])*int(data[z][m])
                    sumuu=sumuu+int(data[i][m])**2
                    sumvv=sumvv+int(data[z][m])**2
            #Заносим окончательный результат в массив метрики сходства
            sim.append((z,round((sumuv/(math.sqrt(sumuu)*math.sqrt(sumvv))),3)))
    #Выбираем k=4 ближиайших пользователей
    sim=sorted(sim,key=itemgetter(1),reverse=True)[:4]
    
    final["kNN"][data[i][0]]=[]
    res[data[i][0]]=[]
    
    #Вычисление по формуле расчёта оценки по методу kNN
    for m in range(1,31):
        sumup=0 #Вверхняя сумма
        sumdwn=0 #Нижняя сумма
        if(int(data[i][m])==-1):
            for k,_ in enumerate(sim):
                if(int(data[sim[k][0]][m])!=-1):
                    sumup=sumup+round(sim[k][1]*(int(data[sim[k][0]][m])-srr[sim[k][0]-1]),3)
                    sumdwn=sumdwn+sim[k][1]
            #Финальный расчёт по формуле, округление и приведение к 3 знакам после запятой
            final["kNN"][data[i][0]].append((data[0][m],"%.3f" %(srr[i-1]+round(sumup/sumdwn,3))))
            res[data[i][0]].append((m,data[0][m],"%.3f" %(srr[i-1]+round(sumup/sumdwn,3))))

    
#Задание 2 Контекстная рекомендация

# Основной принцип: Берутся непросмотренные фильмы из результатов задания 1,
# сортируются по убыванию и затем считаются просмотры других пользователей
# дома(параметр1) и в выходные(параметр2), затем происходит сравнение
# приоритет имеет фильм с наибольшей вычисленной оценкой, если хотя бы один из его параметров
# больше следующего фильма то он выбирается в качестве рекомендуемого

final["recommendations"]={}#Словарь результатов
for i in range(1,41):
    # Сортируем массив с результатами из задания 1
    mas=sorted(res[data[i][0]],key=itemgetter(2),reverse=True)
    tmp=[]#массив для хранения промежуточных результатов
    for k,_ in enumerate(mas):
        Kpl=0 #счётчик просмотров дома
        Kday=0 #счётчик просмотров в выходные
        for z in range(1,41):      
            if(cplace[z][mas[k][0]].strip()=="h"):
                Kpl=Kpl+1
            if((cday[z][mas[k][0]].strip()=="Sat") or (cday[z][mas[k][0]].strip()=="Sun")):
                Kday=Kday+1
        tmp.append((Kpl,Kday,mas[k][1],mas[k][2]))
    #проверка и выход
    for k,_ in enumerate(tmp):
        if(k+1<len(tmp)):
            #Само условие
            if((tmp[k][0]>tmp[k+1][0] and tmp[k][1]>0) or (tmp[k][1]>tmp[k+1][1] and tmp[k][0]>0)):
                final["recommendations"][cday[i][0]]=tmp[k][2]
                break

#Вывод в файл
with open('answer.json', 'w', encoding='utf-8') as f:
       json.dump(final, f, ensure_ascii=False, indent=4)